In [1]:
from math import radians, cos, sin, asin, sqrt

EARTH_RADIUS = 6_371 

def haversine_distance(lat1 : float , lon1 : float, lat2 : float, lon2 : float, unit : str = "m") -> float:
    R = EARTH_RADIUS if unit == "km" else EARTH_RADIUS * 1_000
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R * c

In [2]:
import json 
class DeliveryOrder:
    def __init__(self, id : str, origin_lat : float, origin_long : float, dest_lat : float, dest_long : float, weight : int) -> None:
        id : str = id
        weight : int = weight
        start_position : dict = {
            "latitude": origin_lat,
            "longitude": origin_long
        }
        destination_position : dict = {
            "latitude": dest_lat,
            "longitude": dest_long
        }
        order_status : bool = False
        
    def __repr__(self) -> str:
        return json.dumps({
            "id": id,
            "dest_lat": round(destination_position['latitude'],2),
            "dest_long": round(destination_position['longitude'],2),
            "weight": weight
        })

In [3]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        for _ in range(1000):
            func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {(end_time - start_time):.6f} seconds to run 1000 times.")
    return wrapper

In [40]:
# ----------------------------------------------------------------------------------------------

from itertools import combinations

# ---------------------------------------------------------------------------------------------

def closest_order(latitude, longitude, orders : list[DeliveryOrder]) -> DeliveryOrder:
    min_dist = float('inf')
    closest = None
    for order in orders:
        dist = haversine_distance(
            latitude,
            longitude,
            order.destination_position['latitude'],
            order.destination_position['longitude']
        )
        if dist < min_dist:
            min_dist = dist
            closest = order
    return closest

# ---------------------------------------------------------------------------------------------

def generate_path(orders: list[DeliveryOrder], first_order: DeliveryOrder) -> list[DeliveryOrder]:
    if not orders:
        return []
    start_order = first_order
    if start_order not in orders:
        return []
    path = [start_order]
    visited = {start_order.id} 
    current_order = start_order
    while len(path) < len(orders):
        next_order = None
        min_distance = float('inf')
        for order in orders:
            if order.id not in visited:  
                distance = haversine_distance(
                    current_order.destination_position['latitude'], 
                    current_order.destination_position['longitude'], 
                    order.destination_position['latitude'], 
                    order.destination_position['longitude']
                )
                if distance < min_distance:
                    min_distance = distance
                    next_order = order
        if next_order:
            visited.add(next_order.id)  
            path.append(next_order)
            current_order = next_order
        else:
            break 
    return path

# ---------------------------------------------------------------------------------------------

def calculate_travel_distance(path : list[DeliveryOrder]) -> float:
    if len(path) < 2:
        return 0.0
    total_distance = 0.0
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        total_distance += haversine_distance(
            start.destination_position['latitude'], 
            start.destination_position['longitude'], 
            end.destination_position['latitude'], 
            end.destination_position['longitude']
        )
    return total_distance

# ---------------------------------------------------------------------------------------------

def calculate_capacity_level(orders: list[DeliveryOrder], max_capacity: int) -> float:
    total_weight = sum(order.weight for order in orders)
    capacity_level = total_weight / max_capacity
    return min(capacity_level, 1.0)

# ---------------------------------------------------------------------------------------------

def utility(travel_distance : float, velocity : float, capacity_level : float) -> float:
    travel_time = travel_distance / velocity
    time_alpha = 0.1
    time_beta = 0.01
    capacity_alpha = 2.0
    if travel_time <= 0:
        return float('-inf')
    travel_utility = - time_alpha * (1 / travel_time + travel_time * time_beta)
    capacity_utility = - capacity_alpha * (1 - capacity_level)**2
    return travel_utility + capacity_utility

# ---------------------------------------------------------------------------------------------

def combine_orders(orders : list[DeliveryOrder], capacity : int) -> list[list[DeliveryOrder]]:
    all_combinations = []
    for r in range(1, len(orders) + 1):
        all_combinations.extend(combinations(orders, r))
    valid_combinations = []
    for combo in all_combinations:
        if sum(order.weight for order in combo) <= capacity:
            valid_combinations.append(list(combo))
    return valid_combinations

# ---------------------------------------------------------------------------------------------

def best_available_orders(orders: list[DeliveryOrder], latitude: float, longitude: float, capacity: int, velocity: float) -> list[DeliveryOrder]:
    order_sets = combine_orders(orders, capacity)
    best_set = None
    best_utility = float('-inf')
    for order_set in order_sets:
        closest = closest_order(latitude, longitude, order_set)
        distance_closest_order = haversine_distance(
            latitude, 
            longitude, 
            closest.destination_position['latitude'], 
            closest.destination_position['longitude']
        )
        path = generate_path(order_set, closest)
        travel_distance = distance_closest_order + calculate_travel_distance(path)
        capacity_level = calculate_capacity_level(order_set, capacity)
        set_utility = utility(travel_distance, velocity, capacity_level)
        # print ids in order set
        print([order.id for order in order_set], travel_distance, set_utility)
        if set_utility > best_utility:
            best_set = order_set
            best_utility = set_utility
    return best_set

# ----------------------------------------------------------------------------------------------

def best_order_decision(next_orders, position, max_capacity, velocity, available_order_sets, warehouse_positions) -> str:
    winner = None
    drone_utility = float('-inf')
    
    if next_orders:
        orders = next_orders
        closest = closest_order(position["latitude"], position["longitude"], orders)
        distance_closest_order = haversine_distance(
            position["latitude"], 
            position["longitude"], 
            closest.destination_position['latitude'], 
            closest.destination_position['longitude']
        )
        path = generate_path(orders, closest)
        travel_distance = distance_closest_order + calculate_travel_distance(path)
            
        capacity_level = calculate_capacity_level(orders, max_capacity)
        drone_utility = utility(travel_distance, velocity, capacity_level)
    
    for warehouse, orders in available_order_sets.items():
        if next_orders:
            orders += next_orders
        distance_warehouse = haversine_distance(
            position["latitude"], 
            position["longitude"], 
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude']
        )
        closest_to_warehouse = closest_order(
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude'], 
            orders
        )
        distance_warehouse_to_closest_order = haversine_distance(
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude'], 
            closest_to_warehouse.destination_position['latitude'], 
            closest_to_warehouse.destination_position['longitude']
        )
        path = generate_path(orders, closest_to_warehouse)
        travel_distance = distance_warehouse + distance_warehouse_to_closest_order + calculate_travel_distance(path)
            
        orders += next_orders
        capacity_level = calculate_capacity_level(orders, max_capacity)
        new_utility = utility(travel_distance, velocity, capacity_level)
            
        if new_utility > drone_utility:
            winner = warehouse
            drone_utility = new_utility  
    
    return winner

In [13]:
orders_w1 : list[DeliveryOrder] = [
    DeliveryOrder("w1_1", 18.994237, 72.825553, 19.017584, 72.922585, 20),
    DeliveryOrder("w1_2", 18.994237, 72.825553, 19.007584, 72.912585, 5),
    DeliveryOrder("w1_3", 18.994237, 72.825553, 19.007584, 72.912585, 15),
    DeliveryOrder("w1_4", 18.994237, 72.825553, 18.977584, 72.882585, 5),
    DeliveryOrder("w1_5", 18.994237, 72.825553, 19.017584, 72.922585, 15),
    DeliveryOrder("w1_6", 18.994237, 72.825553, 19.057584, 72.962585, 15),
    DeliveryOrder("w1_7", 18.994237, 72.825553, 18.957584, 72.862585, 20),
    DeliveryOrder("w1_8", 18.994237, 72.825553, 18.997584, 72.902585, 10),
    DeliveryOrder("w1_9", 18.994237, 72.825553, 19.057584, 72.962585, 10),
    DeliveryOrder("w1_10",18.994237, 72.825553, 18.937584, 72.842585, 10)
]

orders_w2 : list[DeliveryOrder] = [
    DeliveryOrder("w2_1", 18.927584, 72.832585, 19.037584, 72.942585, 5),
    DeliveryOrder("w2_2", 18.927584, 72.832585, 19.037584, 72.942585, 5),
    DeliveryOrder("w2_3", 18.927584, 72.832585, 18.947584, 72.852585, 5),
    DeliveryOrder("w2_4", 18.927584, 72.832585, 18.947584, 72.852585, 5),
    DeliveryOrder("w2_5", 18.927584, 72.832585, 18.977584, 72.882585, 20),
    DeliveryOrder("w2_6", 18.927584, 72.832585, 19.057584, 72.962585, 20),
    DeliveryOrder("w2_7", 18.927584, 72.832585, 18.957584, 72.862585, 10),
    DeliveryOrder("w2_8", 18.927584, 72.832585, 18.967584, 72.872585, 5),
    DeliveryOrder("w2_9", 18.927584, 72.832585, 19.017584, 72.922585, 15),
    DeliveryOrder("w2_10",18.927584, 72.832585, 18.937584, 72.842585, 10),
]


In [37]:
w1_position = {
    "latitude": 18.994237, 
    "longitude": 72.825553
}
best_w1_orders = best_available_orders(orders_w1, w1_position['latitude'], w1_position['longitude'], 20, 20)
print([order for order in best_w1_orders])

['w1_1'] 10526.425869824223 -0.5265112915042817
['w1_2'] 9269.798626034511 -1.5887056857189124
['w1_3'] 9269.798626034511 -0.5887056857189125
['w1_4'] 6276.065026615798 -1.43912192234384
['w1_5'] 10526.425869824223 -0.6515112915042817
['w1_6'] 16034.846128957206 -0.9268670348040863
['w1_7'] 5636.835308220503 -0.28219657442896534
['w1_8'] 8107.648665074739 -0.9056291138984738
['w1_9'] 16034.846128957206 -1.3018670348040864
['w1_10'] 6549.192113912366 -0.8277649868732069
['w1_2', 'w1_3'] 9269.798626034511 -0.46370568571891246
['w1_2', 'w1_4'] 10867.05096075014 -1.0435365906086909
['w1_2', 'w1_5'] 10800.040452017578 -0.5401872070924452
['w1_2', 'w1_6'] 16920.573035347676 -0.8461468510739807
['w1_2', 'w1_8'] 9637.933916094702 -0.6071042091595071
['w1_2', 'w1_9'] 16920.573035347676 -0.9711468510739807
['w1_2', 'w1_10'] 17262.099480495555 -0.9882208347868051
['w1_3', 'w1_4'] 10867.05096075014 -0.5435365906086908
['w1_4', 'w1_5'] 12397.292769427773 -0.6200259640153761
['w1_4', 'w1_6'] 18517.8

In [38]:
w2_position = {
    "latitude": 18.927584, 
    "longitude": 72.832585
}
best_w2_orders = best_available_orders(orders_w2, w2_position['latitude'], w2_position['longitude'], 20, 20)
print([order for order in best_w2_orders])

['w2_1'] 16834.090663220275 -1.9668233396971084
['w2_2'] 16834.090663220275 -1.9668233396971084
['w2_3'] 3061.1340136861536 -1.2787100533278242
['w2_4'] 3061.1340136861536 -1.2787100533278242
['w2_5'] 7652.510173140085 -0.3828868608087492
['w2_6'] 19894.269594418027 -0.9948140111825119
['w2_7'] 4591.636071833539 -0.7300173781818037
['w2_8'] 6122.0947994924845 -1.4314314255398601
['w2_9'] 13773.737657079213 -0.813832086723077
['w2_10'] 1530.5886485689216 -0.5778361192393472
['w2_1', 'w2_2'] 16834.090663220275 -1.3418233396971084
['w2_1', 'w2_3'] 16834.090947883105 -1.3418233539282407
['w2_1', 'w2_4'] 16834.090947883105 -1.3418233539282407
['w2_1', 'w2_7'] 16834.09104290054 -0.9668233586784422
['w2_1', 'w2_8'] 16834.09110633209 -1.3418233618495718
['w2_1', 'w2_9'] 16834.09094856513 -0.8418233539623373
['w2_1', 'w2_10'] 16834.090821312206 -0.9668233476005892
['w2_2', 'w2_3'] 16834.090947883105 -1.3418233539282407
['w2_2', 'w2_4'] 16834.090947883105 -1.3418233539282407
['w2_2', 'w2_7'] 168

In [46]:
next_orders = []
max_capacity = 20
velocity = 20
available_order_sets = {
    "w1": best_w1_orders,
    "w2": best_w2_orders
}
warehouse_positions = {
    "w1": {
        "latitude": 18.994237, 
        "longitude": 72.825553
    },
    "w2": {
        "latitude": 18.927584, 
        "longitude": 72.832585
    }
}
position = {
    "latitude": warehouse_positions["w2"]["latitude"],
    "longitude": warehouse_positions["w2"]["longitude"]
}

winner = best_order_decision(next_orders, position, max_capacity, velocity, available_order_sets, warehouse_positions)

print(f"Winner: {winner}")
available_order_sets[winner]

Winner: w2


[{"id": "w2_3", "dest_lat": 18.95, "dest_long": 72.85, "weight": 5},
 {"id": "w2_4", "dest_lat": 18.95, "dest_long": 72.85, "weight": 5},
 {"id": "w2_10", "dest_lat": 18.94, "dest_long": 72.84, "weight": 10}]